# Implementacion Pedagogica de Algoritmos 
A continuacion vemos la implementacion de los algoritmos vistos en clase. Estos son:
* Steepest Descent
* Newton
* Cuasi-Newton


## Parametros de los algoritmos iterativos
* toleracia (abs_tol)
$$\frac{|x_{k+1}-x_k|}{|x_k|} < \epsilon$$

In [1]:
from tqdm.notebook import tqdm
import numpy as np

In [3]:
def backtrack(f, x_k, grad_k, p_k, imprimir=False, max_tries=100):
    # Algoritmos de busqueda de longitud de paso
    # Tambien llamado algoritmo de Backtracking

    ## ENTRADAS
    # f la funcion objetivo
    # x_k el valor actual de la iteracion
    # grad_k el valor del gradiente en x_k
    # p_k la direccion de mejora
    # parametros
    
    # longitud de paso default
    alfa = 1.0
    # factor de contraccion
    rho = 0.9
    # Condicion (1) de Wolfe
    c_1 = 10e-4

    Sentinela = 0
    #print(f"Wolfe condition terms: {f(x_k + alfa*p_k)}  {f(x_k) + c_1*alfa*grad_k.dot_product(p_k)}")
    while (f(x_k + alfa*p_k) > f(x_k) + c_1*alfa*grad_k.dot_product(p_k) and 
           Sentinela < max_tries):
        if imprimir: #
            print(f"{alfa=}")
        alfa = rho*alfa
        Sentinela += 1
    if imprimir:
        print(f"{Sentinela=}")
    return alfa    

### Funciones cuadraticas
Tienen la forma:
$$ f(x) = \frac12 x^T Q x - b^T x$$
Donde $Q$ es una matrix definida positiva,

El gradiente es: $\nabla f(x) = Qx - b$

El hessiano: $\nabla^2 f(x) = Q$

La longitud de paso Optima para la iteracion $k$ es:
$$\alpha_k = \frac{\nabla f_k^T \nabla f_k}{{\nabla f_k^T Q \nabla f_k}}$$

In [6]:
# Generamos funciones cuadraticas de pruebas
# generamos una matrix de numeros enteros de 3X3
dim = 3
M = random_matrix(ZZ, dim, dim)
# M^TM siempre definida semipositiva
Q = M.transpose()*M
#Q = identity_matrix(3)
print(f"Los Val. Propios de Q son {Q.eigenvalues()}")
b = vector(range(dim))
#b = vector([1,1,1])
min_f = vector(RR, (Q.inverse()*b))
print(f"El minimo de f esta en {min_f}")
# funciones lambda = funciones anonimas (inline en C++)
# np es numpy
cuad = lambda x: 1/2*x*(Q*x) - b*x
grad_cuad = lambda x: Q*x - b
hess_cuad = lambda x: Q

Los Val. Propios de Q son [1.179678806250474?, 2.471162835631458?, 12.34915835811807?]
El minimo de f esta en (-0.166666666666667, 0.166666666666667, 1.16666666666667)


### La Funcion de Rosenbrock
Es una funcion de prueba muy usada en optimizacion numerica, especialmente en el analisis de algoritmos iterativos https://en.wikipedia.org/wiki/Rosenbrock_function

La version mas comun en dos dimensiones es:
$$f(x,y) = (1 - x)^2 + 100(y-x^2)^2$$
Tiene un minimo global en (1,1)

In [13]:
# version simbolica
var('x y')
Rosen(x,y) = (1 - x)^2 + 100*(y - x^2)^2
show(Rosen.gradient())
show(Rosen.hessian())
HessianoRosen = Rosen.hessian()(1,1)

(x, y) |--> (400*(x^2 - y)*x + 2*x - 2, -200*x^2 + 200*y)

[(x, y) |--> 1200*x^2 - 400*y + 2               (x, y) |--> -400*x]
[              (x, y) |--> -400*x                  (x, y) |--> 200]

In [19]:
prod([N(_) for _ in HessianoRosen.eigenvalues()])

399.999999999989

In [20]:
def rosen(x):
    ## funcion accesorio para en los algoritmos de optimizacion
    return (1 - x[0])^2 + 100*(x[1] - x[0]^2)^2

Grad_Rosen = Rosen.gradient()
grad_rosen = lambda x: Grad_Rosen(x[0], x[1])

Hess_Rosen = Rosen.hessian()
hess_rosen = lambda x: Hess_Rosen(x[0], x[1])
    
print(rosen(vector([1,1])))
print(grad_rosen(vector([1,1])))
print(hess_rosen(vector([1,1])))

0
(0, 0)
[ 802 -400]
[-400  200]


## Steepest Descent
Es algoritmo mas basico, solo usa el gradiente de la siguiente manera:
$$p_k = -\nabla f(x_k)$$
y luego de escoger la longitud de paso $a_k$ (con algun algoritmo como backtracking) se encuentra el termino siguiente $x_{k+1} = x_k + \alpha_k p_k$.

In [23]:
# Implementamos el metodo del Steepest Descent
# punto inicial para comenzar la iteracion
x_0 = vector([2,2.5,4])
# placeholder para x_k
# el vector que va a llevar las iteraciones
x_k = x_0
contador = 0
MAX_ITER = 1_000
abs_tol = 1e-6
error_ajustado = 1
#pbar = tqdm(total = int(MAX_ITER + 1))
while (contador < MAX_ITER and
      error_ajustado > abs_tol):
    grad_k = grad_cuad(x_k)
    p_k = -grad_cuad(x_k)
    alpha = backtrack(cuad, vector(x_k), vector(grad_k), vector(p_k), imprimir = False)
    # en este punto perdemos la informacion de la iteracion anterior
    x_k1 = x_k - alpha*grad_cuad(x_k)
    error_ajustado = np.linalg.norm(x_k1 - x_k)/np.linalg.norm(x_k)
    x_k = x_k1
    print_line = [repr(N(x_i, digits=3)) for x_i in x_k]
    #print(f"{print_line} con {alpha=}")
    contador += 1
#    pbar.update(int(1))
#pbar.close()
print(f"{print_line} con {alpha=:.4f}, {contador=}, {error_ajustado=:.8f}")

['-0.167', '0.167', '1.17'] con alpha=0.1501, contador=98, error_ajustado=0.00000094


In [26]:
# Si queremos usar la interface de Optimizacion de SAGE 
np_cuad = lambda x: cuad(vector(x))
# La funcion minimize viene en SAGE
# la tolerancia default es 10e-6
minimize(np_cuad, np.array(x_0), algorithm='bfgs', verbose=True)

Optimization terminated successfully.
         Current function value: -1.250000
         Iterations: 7
         Function evaluations: 32
         Gradient evaluations: 8


(-0.1666666195843882, 0.1666667827759691, 1.1666667974507066)

## Algoritmo BFGS
Es el principal algoritmo de tipo Cuasi-Newton

Tiene las siguientes partes:
$$p_k = -H_k \nabla f_k$$

Informalmente, $H_k$ se debe parecer a $(\nabla^2 f_k)^{-1}$
$$H_{k+1} = (Id - \rho_k s_k y_k^T)H_k(Id - \rho_k y_k s_k^T) + \rho_k s_k s_k^T$$
donde $$\rho_k = 1/y_k^Ts_k$$ y $$s_k = x_{k+1} - x_k$$, $$y_k = \nabla f_{k+1} - \nabla f_k$$

In [28]:
# Funciones Quasi-Newton
def get_H_k1(f, grad_f, x_k, x_k1, H_k):
    # Funcion para encontrar la actualizacion de H_k
    y_k = grad_f(x_k1) - grad_f(x_k)
    s_k = x_k1 - x_k
    dim = len(x_k)
    Id = identity_matrix(dim)
    # note que sumamos 1 en denominador
    # para mejorar la estabilidad numerica
    # otra estrategia usada es: max(y_k*s_k, 1)
    rho_k = 1/(y_k.dot_product(s_k) + 1)
    skyk = rho_k*s_k.column()*y_k.row()
    yksk = rho_k*y_k.column()*s_k.row()
    sksk = rho_k*s_k.column()*s_k.row()
    #print(rho_k)
    return (Id - rho_k*skyk)*H_k*(Id - rho_k*yksk) + rho_k*sksk

def theta(p_k, grad_k):
    # Encuentra el cos(theta) donde theta es el angulo interior
    return p_k*grad_k/p_k.norm()/grad_k.norm()

In [30]:
# Implementamos el metodo  Quasi-Newton
# punto inicial para comenzar la iteracion
x_0 = vector([2,2.5,4])
# placeholder para x_k
# el vector que va a llevar las iteraciones
x_k = x_0
# Usamos H_0 = Id
# No es la mejor forma pero es la mas sencilla
H_k = identity_matrix(dim)
contador = 0
MAX_ITER = 1000
abs_tol = 1e-6
error_ajustado = 1
#pbar = tqdm(total = int(MAX_ITER + 1))
while (contador < MAX_ITER and
      error_ajustado > abs_tol):
    grad_k = grad_cuad(x_k)
    # Este es el nuevo vector de direccion de mejora
    p_k = -H_k*grad_k
    #p_k = -Q.inverse()*grad_k
    alpha = backtrack(cuad, x_k, grad_k, p_k, imprimir = False)
    
    #print(f"|grad|={grad_k.norm():.3f}, |p_k| = {p_k.norm():.3f} , alpha={alpha:.3f}, theta={theta(grad_k,p_k):.3f}")
    # encontramos la nueva iteracion
    x_k1 = x_k + alpha*p_k
    # actualizamos H_k
    H_k = get_H_k1(cuad, grad_cuad, x_k, x_k1, H_k)
    error_ajustado = np.linalg.norm(x_k1 - x_k)/np.linalg.norm(x_k)    
    # en este punto perdemos la informacion de la iteracion anterior
    x_k = x_k1
    print_line = [repr(N(x_i, digits=3)) for x_i in x_k]
    #print(f"{print_line} con {alpha=}")
    contador += 1
#    pbar.update(int(1))
#pbar.close()
print(f"Sols: {print_line} \n {alpha=}, \n {contador=}, \n {error_ajustado=}")

Sols: ['-0.167', '0.167', '1.17'] 
 alpha=1.00000000000000, 
 contador=32, 
 error_ajustado=2.5764428439839517e-07


## Newton??